Only need to add the function PeekingTimes

In [1]:
import numpy as np

results_array = np.load('results_array.npy')

zones_order = ['F1', 'F2', 'F3', 'F4', 'CL', 'M', 'OB', 'CR', 'OT']

def arm_entry(results_array, zones_order):
    # Arm entries version from 20130829 10:10 am. Doesn't capture extra events anymore. Seems accurate now.
    # each list of arm entries contains when a mouse entered the arm
    arm_entries = [[], [], [], [], [], [], [], [], []] # the number of lists = the number of zones
    for frame in range(len(results_array) - 1): # len(results_array)=9000
        # Compare each arm, one by one to the middle
        for zone in [4, 6, 7, 8]:  # 4,6,7,8 are the arms
            if results_array[
                frame, 5] > 0:  # 5 is the middle area. Arm antries can only occur if a mouse was present in the middle.
                if results_array[frame, zone] < results_array[frame, 5] and np.argmax(
                        results_array[frame + 1]) == zone:
                    arm_entries[zone].append(frame + 1)
    tot_arm_entries = dict(zip(zones_order, [len(arm) for arm in arm_entries])) # how many times the mouse entered each arm
    return arm_entries, tot_arm_entries

arm_entries, tot_arm_entries = arm_entry(results_array, zones_order)

In [2]:
# if a mouse leaves an arm then goes back without entering any other arm in a threshold of frames(e.g., threshold = 150(5 seconds))
# then count it as one time of peeking
def PeekingTimes(arm_entries, threshold):
    times = dict()
    arms = [4, 6, 7, 8]
    for arm in arms:
        times[arm] = 0
    
    entries = sorted([entry for zones in arm_entries for entry in zones])
    arm_entries_tuple = [(entry, zone) for zone, subentries in enumerate(arm_entries) for entry in subentries]
    entries_dict = dict(arm_entries_tuple)
    
    for k in range(len(entries)-1):
        if entries[k+1]-entries[k]<threshold and entries_dict[entries[k]]==entries_dict[entries[k+1]]:
            times[entries_dict[entries[k]]] += 1
            
    return times

In [3]:
PeekingTimes(arm_entries, 150)

{4: 1, 6: 0, 7: 2, 8: 0}